In [1]:
import pandas as pd
import numpy as np

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
df = pd.read_csv("diffusion_db_unaltered.csv")

df.info()

In [ ]:
df.head()

In [ ]:
df_part = df.sample(n=100000, random_state=42)

In [ ]:
df_part.info()

In [ ]:
df[df["prompt_nsfw"] == df["prompt_nsfw"].max()]

In [ ]:
df.iloc[191144]

In [ ]:
df['nsfw'] = (df['image_nsfw'] >= 0.7) | (df['prompt_nsfw'] >= 0.7)
nsfw_t = df[df['nsfw'] >= 0.7]
nsfw_t

In [ ]:
nsfw_f = df[df['nsfw'] < 0.7]
nsfw_f

In [ ]:
prompts_nsfw = df[df['prompt_nsfw'] >= 0.7]
prompts_nsfw[['prompt', 'prompt_nsfw']]

In [ ]:
pnsfw_m = df[df['prompt_nsfw'] >= 0.99]
pnsfw_m[['prompt', 'prompt_nsfw']]

In [ ]:
lists = [
    ["attack", "fight", "kill", "murder", "assault", "punch", "stab", "shoot", "beat", "hurt", "injure", "strike", "poison", "strangle", "violent", "violence", "decapitate", "aggression", "batter", "wound", "torture", "slaughter", "massacre", "assassinate"],
    ["gun", "knife", "sword", "bomb", "bullet", "firearm", "machete", "axe", "crossbow", "caliber", "artillery", "weaponry", "explosive", "sledgehammer", "cannon", "shuriken", "nunchucks", "spear", "trident", "surujin", "whip", "charkram", "sickle", "dagger"],
    ["blood", "death", "bloodshed", "massacre", "war", "destruction", "devastation", "bloodbath", "brutality", "brutal", "cruelty", "terror", "horror", "rage", "hatred", "carnage", "genocide"],
    ["conflict", "terrorize", "uprising", "riot", "warfare", "rebellion", "terrorism", "hostage", "kidnapping", "sacrifice", "war", "trafficking", "execute", "homocide", "bully", "death", "die", "explode", "savage", "vicious"],
]

"attack", "fight", "kill", "murder", "assault", "punch", "stab", "shoot", "beat", "hurt", "injure", "strike", "poison", "strangle", "violent", "violence", "decapitate", "aggression", "batter", "wound", "torture", "slaughter", "massacre", "assassinate", "gun", "knife", "sword", "bomb", "bullet", "firearm", "machete", "axe", "crossbow", "caliber", "blood", "death", "bloodshed", "massacre", "war", "destruction", "devastation", "bloodbath", "brutality", "brutal", "cruelty", "terror", "horror", "rage", "hatred", "carnage", "genocide", "conflict", "terrorize", "uprising", "riot", "warfare", "rebellion", "terrorism", "hostage", "kidnapping", "sacrifice", "war", "trafficking", "execute", "homocide", "bully", "death", "die", "explode", "savage", "vicious", "artillery", "weaponry", "explosive", "sledgehammer", "cannon", "shuriken", "nunchucks", "spear", "trident", "surujin", "whip", "charkram", "sickle", "dagger"

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

def vectorize_prompt(prompt, lists):
    vector = []
    prompt = prompt.lower()
    for i, words in enumerate(lists):
        if any(word in prompt for word in words):
            vector.append(1)
        else:
            vector.append(0)
    return vector

In [ ]:
prompt = "The brutal voilence of war."

vector = vectorize_prompt(prompt, lists)

vector

In [ ]:
def vectorize_dataframe(df_part, lists):
    return df_part['prompt'].apply(lambda x: vectorize_prompt(x, lists))
    
df_part['prompt'] = df_part['prompt'].fillna('').astype(str)

df_part['vector'] = vectorize_dataframe(df_part, lists)

In [ ]:
X = np.array(df_part['vector'].tolist())
y = df_part['prompt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

In [ ]:
new_prompt = "The brutal violence of war"
new_vector = vectorize_prompt(new_prompt, lists) 

prediction = knn.predict([new_vector])
print("Prediction:", "Harmful" if prediction[0] == 1 else "Harmless")